<a href="https://colab.research.google.com/github/LeonelaFaria/algoritimos/blob/main/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-telegram-bot ultralytics vosk nest_asyncio
!apt-get install ffmpeg -y
import nest_asyncio
nest_asyncio.apply()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.0/731.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.1 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=3bf245f5718596dcbd45d374a99c025a1879465e1b0005315ba1eb7c1f1d41ec
  Stored in directory: /root/.cache/pip/wheels/7e/75/5b/e1d5c3756631e4bda806f6cc9640153b39484bb6f7b0b8def3
Successfully built srt
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [2]:
# ======================================================
# IMPORTAÇÕES
# ======================================================
import os
import json
import wave
import tempfile
import subprocess
import nest_asyncio
import numpy as np
from abc import ABC, abstractmethod

from ultralytics import YOLO
from vosk import Model, KaldiRecognizer

from telegram import Update
from telegram.ext import (
    ApplicationBuilder,
    ContextTypes,
    CommandHandler,
    MessageHandler,
    filters
)

nest_asyncio.apply()  # Necessário para ambientes como Google Colab


# ======================================================
# CONFIGURAÇÕES GERAIS (ENCAPSULADAS)
# ======================================================
TOKEN = "8175320546:AAHFe2D8zFYW2vo7rOH-D9BkTQZPJKAVVt4"
CAMINHO_YOLO = "best.pt"
CAMINHO_VOSK = "vosk-model-small-pt-0.3"


# ======================================================
# FUNÇÃO AUXILIAR — ANÁLISE ESTATÍSTICA ROBUSTA (YOLO)
# Usa mediana + IQR para reduzir falsos positivos
# ======================================================
def analise_estatistica_yolo(deteccoes):
    """
    Aplica estatística robusta nas detecções do YOLO.
    Remove outliers e calcula a confiança final.
    """
    agrupado = {}

    # Agrupa por classe
    for d in deteccoes:
        agrupado.setdefault(d["classe"], []).append(d["conf"])

    resultado = []

    for classe, confs in agrupado.items():
        confs = np.array(confs)

        # Cálculo do IQR (Intervalo Interquartil)
        q1 = np.percentile(confs, 25)
        q3 = np.percentile(confs, 75)
        iqr = q3 - q1

        # Remove outliers
        confs_validas = confs[
            (confs >= q1 - 1.5 * iqr) &
            (confs <= q3 + 1.5 * iqr)
        ]

        resultado.append({
            "classe": classe,
            "confianca": float(np.median(confs_validas)),
            "amostras": len(confs_validas)
        })

    return resultado


# ======================================================
# CLASSE BASE — BOT TELEGRAM
# Responsável pela comunicação com a API do Telegram
# ======================================================
class BotTelegram(ABC):
    """
    Classe base abstrata para bots do Telegram.
    """

    def __init__(self, app):
        self._app = app  # Encapsulamento do Application

    @abstractmethod
    def registrar(self):
        """
        Registra os handlers específicos de cada bot.
        """
        pass


# ======================================================
# BOT DE IMAGEM — PROCESSAMENTO COM YOLO
# ======================================================
class BotImagem(BotTelegram):

    def __init__(self, app, caminho_modelo):
        super().__init__(app)
        self._modelo = YOLO(caminho_modelo)

    def registrar(self):
        self._app.add_handler(
            MessageHandler(filters.PHOTO, self.processar)
        )

    async def processar(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """
        Processa imagens enviadas pelo usuário.
        """
        status = await update.message.reply_text(
            "📥 Imagem recebida\n🧠 Iniciando análise..."
        )

        # Download da imagem
        foto = await update.message.photo[-1].get_file()
        with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp:
            await foto.download_to_drive(tmp.name)
            caminho_img = tmp.name

        # Execução do YOLO
        await status.edit_text("🤖 Executando YOLO...")
        resultados = self._modelo.predict(
            source=caminho_img,
            conf=0.15,
            verbose=False
        )

        os.remove(caminho_img)

        # Coleta das detecções
        deteccoes = []
        for r in resultados:
            for box in r.boxes:
                deteccoes.append({
                    "classe": r.names[int(box.cls)],
                    "conf": float(box.conf)
                })

        if not deteccoes:
            await status.edit_text("❌ Nenhum objeto identificado.")
            return

        # Análise estatística
        await status.edit_text("📊 Refinando resultados...")
        resultado_final = analise_estatistica_yolo(deteccoes)

        # Resposta ao usuário
        resposta = "🔎 **Resultado da Classificação:**\n\n"
        for d in resultado_final:
            resposta += (
                f"• {d['classe']} — "
                f"{d['confianca']:.1%} "
                f"(amostras: {d['amostras']})\n"
            )

        await status.edit_text(resposta, parse_mode="Markdown")


# ======================================================
# BOT DE ÁUDIO — RECONHECIMENTO DE FALA COM VOSK
# ======================================================
class BotAudio(BotTelegram):

    def __init__(self, app, caminho_modelo):
        super().__init__(app)
        self._model = Model(caminho_modelo)

    def registrar(self):
        self._app.add_handler(
            MessageHandler(filters.VOICE | filters.AUDIO, self.processar)
        )

    async def processar(self, update, context):
        """
        Processa mensagens de áudio e converte fala em texto.
        """
        status = await update.message.reply_text(
            "🎤 Áudio recebido\n⚙️ Preparando transcrição..."
        )

        # Identifica o tipo de áudio
        if update.message.voice:
            file = await update.message.voice.get_file()
            suffix = ".ogg"

        elif update.message.audio:
            mime = update.message.audio.mime_type
            if not mime or not mime.startswith("audio/"):
                await status.edit_text("❌ Arquivo inválido.")
                return
            file = await update.message.audio.get_file()
            suffix = os.path.splitext(update.message.audio.file_name)[1]

        else:
            await status.edit_text("❌ Nenhum áudio detectado.")
            return

        # Salva temporariamente
        temp_audio = tempfile.NamedTemporaryFile(
            delete=False, suffix=suffix
        ).name
        await file.download_to_drive(temp_audio)

        # Converte para WAV padrão
        wav_path = temp_audio.replace(suffix, ".wav")
        subprocess.run([
            "ffmpeg",
            "-i", temp_audio,
            "-ar", "16000",
            "-ac", "1",
            wav_path,
            "-y"
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Reconhecimento de fala
        await status.edit_text("🧠 Reconhecendo fala...")
        wf = wave.open(wav_path, "rb")
        rec = KaldiRecognizer(self._model, wf.getframerate())

        while True:
            data = wf.readframes(4000)
            if not data:
                break
            rec.AcceptWaveform(data)

        texto = json.loads(rec.FinalResult()).get("text", "")

        wf.close()
        os.remove(temp_audio)
        os.remove(wav_path)

        # Resposta ao usuário
        if not texto.strip():
            await status.edit_text(
                "⚠️ Não foi possível reconhecer o áudio."
            )
        else:
            await status.edit_text(
                f"📝 **Transcrição:**\n\n{texto}",
                parse_mode="Markdown"
            )


# ======================================================
# COMANDO /START
# ======================================================
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "🤖 **Bot de Inteligência Artificial**\n\n"
        "📸 Envie uma imagem para classificação\n"
        "🎤 Envie um áudio para transcrição\n\n"
        "YOLO + Vosk\n"
        "POO aplicada com sucesso!",
        parse_mode="Markdown"
    )


# ======================================================
# MAIN — EXECUÇÃO DO BOT
# ======================================================
if __name__ == "__main__":

    app = ApplicationBuilder().token(TOKEN).build()

    app.add_handler(CommandHandler("start", start))

    bot_imagem = BotImagem(app, CAMINHO_YOLO)
    bot_audio = BotAudio(app, CAMINHO_VOSK)

    bot_imagem.registrar()
    bot_audio.registrar()

    print("🚀 Bot em execução")
    app.run_polling(drop_pending_updates=True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


FileNotFoundError: [Errno 2] No such file or directory: 'best.pt'

In [ ]:
8175320546:AAHFe2D8zFYW2vo7rOH-D9BkTQZPJKAVVt4